# Prévisions météorologiques en Australie

# Modélisation avec étape de ré-échantillonage

## Récupération des librairies

In [1]:
# récupération du package imblearn
!pip install --upgrade imblearn

In [2]:
# récupération des librairies
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import time, datetime
import warnings
warnings.filterwarnings("ignore")

from sklearn import neighbors
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler

from imblearn.under_sampling import RandomUnderSampler
from imblearn.metrics import classification_report_imbalanced

## Modèle de classification par apprentissage supervisé

### Préparation du jeu de données et paramètrage

In [3]:
# récuperation et aperçu du jeu de données après prétraitement des données, après l'étape de classification des villes 
# en fonction du climat et après sélection des variables les plus pertinentes selon la méthode SelectKBest()
df = pd.read_csv('../data/weatherAUS_preprocessed.csv', index_col=0)
df.head()

,Rainfall,Sunshine,WindGustSpeed,Humidity3pm,Pressure9am,Cloud3pm,Temp3pm,Temp_Delta_MinMax,Humidity_Delta,clim_chaud_humide,clim_méditerranéen,clim_sec,clim_tempéré_froid,RainTomorrow
0,0.6,4.94000,44.0,22.0,1007.7,7.40000,21.8,9.5,-49.0,1,0,0,0,0
1,0.0,7.63054,44.0,25.0,1010.6,4.49925,24.3,17.7,-19.0,1,0,0,0,0
2,0.0,7.96000,46.0,30.0,1007.6,2.00000,23.2,12.8,-8.0,1,0,0,0,0
3,0.0,7.63054,24.0,16.0,1017.6,4.49925,26.5,18.8,-29.0,1,0,0,0,0
4,1.0,0.90000,41.0,33.0,1010.8,8.00000,29.7,14.8,-49.0,1,0,0,0,0


In [4]:
# séparation des données en une matrice de données et en un vecteur cible
data = df.drop('RainTomorrow', axis=1)
target = df['RainTomorrow']

In [5]:
# division des données en un ensemble d'entraînement et un ensemble de test
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=123)

In [6]:
# undersampling des données
rUs = RandomUnderSampler()
X_train, y_train = rUs.fit_resample(X_train, y_train)

In [7]:
# distribution de la variable cible ré-échantillonnée dans l'ensemble d'entraînement
print("Cardinalité des classes de la variable cible ré-échantillonnée:", dict(pd.Series(y_train).value_counts()))

Cardinalité des classes de la variable cible ré-échantillonnée: {0: 21854, 1: 21854}


In [8]:
# distribution de la variable cible ré-échantillonnée dans l'ensemble d'entraînement
y_train.value_counts(normalize=True)

0    0.5
1    0.5
Name: RainTomorrow, dtype: float64

In [9]:
# distribution de la variable cible dans l'ensemble de test
print("Cardinalité des classes de la variable cible dans l'ensemble de test:", 
      dict(pd.Series(y_test).value_counts()))

Cardinalité des classes de la variable cible dans l'ensemble de test: {0: 32890, 1: 9347}


In [10]:
# distribution de la variable cible dans l'ensemble de test
y_test.value_counts(normalize=True)

0    0.778701
1    0.221299
Name: RainTomorrow, dtype: float64

In [11]:
# standardisation des données
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Régression Logistique


In [12]:
# définition du modèle
clf_lr = LogisticRegression(class_weight='balanced', C=1.0, solver='sag')

# entraînement du modèle
clf_lr.fit(X_train_scaled, y_train)

LogisticRegression(class_weight='balanced', solver='sag')

In [13]:
# prédictions du modèle à l'aide d'un seuil de probabilités adapté pour mieux distinguer les classes
y_probas_test = clf_lr.predict_proba(X_test_scaled)
y_pred_test = (y_probas_test[:,1] >= 0.48).astype(bool)

# vérification de l'underfitting et overfitting
print('Train Data Score: {}'.format(clf_lr.score(X_train_scaled, y_train)))
print('Test Data Score: {}'.format(clf_lr.score(X_test_scaled, y_test)))

# matrice de confusion
print('\n Matrice de confusion:')
print('\n', pd.crosstab(y_test, y_pred_test, rownames=['Réalités'], colnames=['Prédictions']))

# affichage du rapport de classification 
print('\n', classification_report(y_test, y_pred_test))

# affichage du rapport de classification spécifique aux jeux de données déséquilibrés
print('\n', classification_report_imbalanced(y_test, y_pred_test))

Train Data Score: 0.7738857874988561
Test Data Score: 0.7790089258233303

 Matrice de confusion:

 Prédictions  False  True
Réalités                
0            25318  7572
1             2098  7249

               precision    recall  f1-score   support

           0       0.92      0.77      0.84     32890
           1       0.49      0.78      0.60      9347

    accuracy                           0.77     42237
   macro avg       0.71      0.77      0.72     42237
weighted avg       0.83      0.77      0.79     42237


                    pre       rec       spe        f1       geo       iba       sup

          0       0.92      0.77      0.78      0.84      0.77      0.60     32890
          1       0.49      0.78      0.77      0.60      0.77      0.60      9347

avg / total       0.83      0.77      0.77      0.79      0.77      0.60     42237



- `0.77 < accuracy, sensibilité, spécificité, moyenne géométrique < 0.78`   

La classe positive est caractérisée par:
- une précision très faible, ce qui signifie que la classe n'est pas bien détectée
- un rappel élevé ce qui signifie que lorsque la classe est détectée, le modèle est fiable
- `f1-score = 0.6`
- `f1-macro = 0.72`

### Interprétabilité du modèle

L'Analyse en Composantes Principales (PCA) permet de réduire le nombre de dimensions d'un jeu de données décrit par un grand nombre de variables. Cela permet une visualisation simplifiée des résultats du modèle.

In [14]:
# calcul de la PCA en ne conservant que deux axes
pca = PCA(n_components=2)

In [15]:
# récupération de la projection du jeu de données d'entraînement sur les deux axes principaux de la PCA 
X_new = pca.fit_transform(X_train_scaled)

xs = X_new[:,0]
ys = X_new[:,1]

In [16]:
# création d'un DataFrame pour stocker les projections standardisées sur chaque axe de la PCA 
scale_x = 1.0/(xs.max() - xs.min())
scale_y = 1.0/(ys.max() - ys.min())

principalDf = pd.DataFrame({'PC1': xs*scale_x, 'PC2': ys*scale_y})

In [17]:
# récupération des vecteurs propres et valeurs propres associées à la PCA
coeff = pca.components_.transpose()

In [18]:
# prédiction du modèle sur le jeu d'entraînement
y_train_pred = clf_lr.predict(X_train_scaled)

In [19]:
# regroupement des projections standardisées sur chaque axe de la PCA et des prédictions du modèle 
# sur le jeu d'entraînement 
finalDF = pd.concat([principalDf, pd.Series(y_train_pred, name='RainTomorrow')], axis=1)

In [20]:
# aperçu du DataFrame
finalDF.head()

,PC1,PC2,RainTomorrow
0,0.042264,-0.248847,0
1,0.178094,0.154613,0
2,0.179986,0.336754,0
3,0.149442,0.379380,0
4,0.235349,-0.172139,0


In [21]:
# visualisation des résultats de la PCA la projection des individus sur deux composantes sélectionnées et la 
# corrélation des variables avec ces composantes 
plt.figure(figsize=(10, 10))
sns.scatterplot(x='PC1', y='PC2', hue='RainTomorrow', data=finalDF, alpha=0.5)
for i in range(X_train_scaled.shape[1]):
    plt.arrow(0, 0, coeff[i,0]*1.05, coeff[i,1]*1.05, color='k', alpha=0.5, head_width=0.01)
    plt.text(coeff[i,0]*1.05, coeff[i,1]*1.05, X_train.columns[i], color='k', fontsize="small")
plt.xlim(-0.6, 0.8)
plt.ylim(-0.6, 0.8)
plt.title("PCA", fontsize="small")
plt.tick_params(labelsize=7);

Ce biplot nous permet de visualiser les classes de précipitations créées par notre modèle en fonction des variables explicatives dans le jeu d'entraînement. 

On observe que lorsqu'il pleut le lendemain, en général le temps est nuageux à 15h, l'humidité est importante à 15h, il y a des précipitations et la variation de l'humidité sur la journée est élevée. 

En revanche, lorsqu'il ne pleut pas le lendemain, la journée bénéficie d'un ensoleillement important, d'un climat sec, d'une forte température à 15h et la variation des températures sur la journée est élevée.

In [22]:
# conversion de l'ensemble d'entraînement standardisé en DataFrame
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)

In [23]:
# affichage de l'importance des variables dans le jeu de données d'entraînement
pd.Series(clf_lr.coef_[0], X_train_scaled.columns).sort_values(ascending=False).plot(kind='barh', figsize=(10,4), fontsize="small")
plt.tick_params(labelsize=7);

Les coefficients du modèle de régression logistique permettent d'avoir un aperçu des relations faites par le modèle entre chaque variable explicative et la variable cible. 
Dans notre cas, la variable "Humidity3pm" est élévée cela signifie que plus l'humidité à 15h est importante et plus la probabilité qu'il pleuve le lendemain est importante.